In [7]:
# Import our dependencies
import pandas as pd
import matplotlib.pyplot as plt
from geopy import Nominatim
%matplotlib inline
import googlemaps
from config import API_KEY

In [8]:
# ****** DATASET 1 : THE FIRE DATA *******

#  Import and read the charity_data.csv.
import pandas as pd 
fire_df = pd.read_csv("Data/California_Fire_Incidents.csv")
fire_df[fire_df["CountyIds"] == '6,11,17,23']["Name"]

1054    Ranch Fire (Mendocino Complex)
1055    Ranch Fire (Mendocino Complex)
1056    Ranch Fire (Mendocino Complex)
1057    Ranch Fire (Mendocino Complex)
Name: Name, dtype: object

In [9]:
fire_df = fire_df.drop(["Active", 
"AdminUnit", 
"AirTankers",
"CanonicalUrl", 
"ConditionStatement", 
"ControlStatement",
"CrewsInvolved",
"Dozers",
"Engines",
"Helicopters",
"Extinguished",
"Fatalities",
"Featured",
"Final",
"FuelType",
"Injuries",
"Location",
# "Name",
"PercentContained",
"PersonnelInvolved",                        
"Public",
"SearchDescription",
"SearchKeywords",
"Status",
"StructuresDamaged",
"StructuresDestroyed",
"StructuresEvacuated",
"StructuresThreatened",
"UniqueId",
"Updated",
"WaterTenders"
], axis=1)
fire_df.head()

,AcresBurned,ArchiveYear,CalFireIncident,Counties,CountyIds,Latitude,Longitude,MajorIncident,Name,Started
0,257314.0,2013,True,Tuolumne,55,37.857000,-120.086000,False,Rim Fire,2013-08-17T15:25:00Z
1,30274.0,2013,True,Los Angeles,19,34.585595,-118.423176,False,Powerhouse Fire,2013-05-30T15:28:00Z
2,27531.0,2013,True,Riverside,33,33.709500,-116.728850,False,Mountain Fire,2013-07-15T13:43:00Z
3,27440.0,2013,False,Placer,31,39.120000,-120.650000,False,American Fire,2013-08-10T16:30:00Z
4,24251.0,2013,True,Ventura,56,0.000000,0.000000,True,Springs Fire,2013-05-02T07:01:00Z


In [10]:
# LOOK AT UNDECIDED COUNTIES
[(column, fire_df[column].nunique()) for column in fire_df]

[('AcresBurned', 634),
 ('ArchiveYear', 7),
 ('CalFireIncident', 2),
 ('Counties', 59),
 ('CountyIds', 79),
 ('Latitude', 1441),
 ('Longitude', 1448),
 ('MajorIncident', 2),
 ('Name', 1193),
 ('Started', 1597)]

In [11]:
[(column, fire_df[column].unique()) for column in fire_df[["Counties","CountyIds"]]]

[('Counties',
  array(['Tuolumne', 'Los Angeles', 'Riverside', 'Placer', 'Ventura',
         'Fresno', 'Siskiyou', 'Humboldt', 'Tehama', 'Shasta', 'San Diego',
         'Kern', 'Sonoma', 'Contra Costa', 'Butte', 'Tulare',
         'Santa Barbara', 'Mariposa', 'Monterey', 'El Dorado',
         'San Bernardino', 'Plumas', 'Modoc', 'San Luis Obispo', 'Madera',
         'Inyo', 'Napa', 'San Benito', 'San Joaquin', 'Lake', 'Alameda',
         'Glenn', 'Yolo', 'Sacramento', 'Stanislaus', 'Solano', 'Merced',
         'Mendocino', 'Lassen', 'Amador', 'Yuba', 'Nevada', 'Santa Clara',
         'Calaveras', 'San Mateo', 'Orange', 'Colusa', 'Trinity',
         'Del Norte', 'Mono', 'Alpine', 'Sutter', 'Kings', 'Sierra',
         'Santa Cruz', 'Marin', 'Mexico', 'State of Oregon',
         'State of Nevada'], dtype=object)),
 ('CountyIds',
  array(['55', '19', '33', '31', '56', '10', '47', '12', '52', '45', '37',
         '15,56', '49', '7', '4', '54', '42', '22', '33,', '27', '15', '9',
         '3

In [12]:
fire_df[fire_df["CountyIds"] == '6,11,17,23']

,AcresBurned,ArchiveYear,CalFireIncident,Counties,CountyIds,Latitude,Longitude,MajorIncident,Name,Started
1054,410203.0,2018,True,Colusa,"6,11,17,23",39.243283,-123.103367,True,Ranch Fire (Mendocino Complex),2018-07-27T12:05:00Z
1055,410203.0,2018,True,Glenn,"6,11,17,23",39.243283,-123.103367,True,Ranch Fire (Mendocino Complex),2018-07-27T12:05:00Z
1056,410203.0,2018,True,Lake,"6,11,17,23",39.243283,-123.103367,True,Ranch Fire (Mendocino Complex),2018-07-27T12:05:00Z
1057,410203.0,2018,True,Mendocino,"6,11,17,23",39.243283,-123.103367,True,Ranch Fire (Mendocino Complex),2018-07-27T12:05:00Z


In [13]:
fire_df[fire_df["CountyIds"] == '6']

,AcresBurned,ArchiveYear,CalFireIncident,Counties,CountyIds,Latitude,Longitude,MajorIncident,Name,Started
841,70.0,2017,True,Colusa,6,39.012240,-122.201270,True,Sand Fire,2017-07-17T13:17:00Z
1218,75.0,2018,True,Colusa,6,39.297600,-122.512012,True,Stony Fire,2018-07-04T13:14:00Z
1599,20.0,2019,True,Colusa,6,39.088333,-122.309167,True,Spring Fire,2019-08-14T12:52:00Z


In [14]:
# Since CountyIDs is clearly causing the data to be skewed, we remove that column:
fire_df = fire_df.drop(["CountyIds"], axis=1)

In [15]:
# EXPORT FIRE TO CSV

# fire_df = fire_df.dropna()
fire_df.to_csv('Clean_Data/fire.csv')

In [16]:
# ****** DATASET 2 : THE ENVIRONMENTAL DATA *******

env_df = pd.read_csv("Data/all_env_conditions.csv")
env_df = env_df.drop(["Wind Run (miles)", "Target", "CIMIS Region"], axis=1)
env_df_2 = pd.read_csv("Data/2013_2017_monthly_env_conditions.csv")
env_df_2 = env_df_2.drop(["Index", "CIMIS Region"], axis=1)

env_df = env_df.append(env_df_2)

env_df.head()

,Stn Id,Stn Name,Date,ETo (in),Precip (in),Sol Rad (Ly/day),Avg Vap Pres (mBars),Max Air Temp (F),Min Air Temp (F),Avg Air Temp (F),Max Rel Hum (%),Min Rel Hum (%),Avg Rel Hum (%),Dew Point (F),Avg Wind Speed (mph),Avg Soil Temp (F)
0,2,FivePoints,1/1/2018,0.06,0.00,219.0,7.3,63.4,35.3,47.8,82.0,46.0,65.0,36.6,3.3,51.1
1,2,FivePoints,1/2/2018,0.04,0.00,127.0,7.4,59.8,37.7,47.2,80.0,52.0,67.0,36.7,3.1,51.3
2,2,FivePoints,1/3/2018,0.04,0.00,125.0,8.4,61.1,37.3,49.9,79.0,49.0,68.0,39.9,4.5,51.3
3,2,FivePoints,1/4/2018,0.07,0.01,219.0,11.6,69.2,48.7,56.8,94.0,52.0,74.0,48.5,5.8,53.0
4,2,FivePoints,1/5/2018,0.07,0.00,239.0,12.7,73.8,47.5,59.8,94.0,49.0,72.0,50.8,4.2,54.4


In [17]:
# ADD THE "YEAR" COLUMN TO THE DATABASE
env_df["Year"] = pd.DatetimeIndex(env_df["Date"]).year

In [18]:
env_df.head()

,Stn Id,Stn Name,Date,ETo (in),Precip (in),Sol Rad (Ly/day),Avg Vap Pres (mBars),Max Air Temp (F),Min Air Temp (F),Avg Air Temp (F),Max Rel Hum (%),Min Rel Hum (%),Avg Rel Hum (%),Dew Point (F),Avg Wind Speed (mph),Avg Soil Temp (F),Year
0,2,FivePoints,1/1/2018,0.06,0.00,219.0,7.3,63.4,35.3,47.8,82.0,46.0,65.0,36.6,3.3,51.1,2018
1,2,FivePoints,1/2/2018,0.04,0.00,127.0,7.4,59.8,37.7,47.2,80.0,52.0,67.0,36.7,3.1,51.3,2018
2,2,FivePoints,1/3/2018,0.04,0.00,125.0,8.4,61.1,37.3,49.9,79.0,49.0,68.0,39.9,4.5,51.3,2018
3,2,FivePoints,1/4/2018,0.07,0.01,219.0,11.6,69.2,48.7,56.8,94.0,52.0,74.0,48.5,5.8,53.0,2018
4,2,FivePoints,1/5/2018,0.07,0.00,239.0,12.7,73.8,47.5,59.8,94.0,49.0,72.0,50.8,4.2,54.4,2018


In [19]:
active_stations_df = pd.read_csv("Data/Active_Fire_Stations.csv")
active_stations_df = active_stations_df.drop(["Status", "Name"], axis=1) 

In [20]:
active_stations_df.head()

,Station ID,County
0,2,Fresno
1,5,Kern
2,6,Yolo
3,7,Fresno
4,12,Butte


In [21]:
env_df = env_df.merge(right=active_stations_df, left_on="Stn Id", right_on="Station ID").drop("Station ID", axis=1)

In [22]:
env_df.head()

,Stn Id,Stn Name,Date,ETo (in),Precip (in),Sol Rad (Ly/day),Avg Vap Pres (mBars),Max Air Temp (F),Min Air Temp (F),Avg Air Temp (F),Max Rel Hum (%),Min Rel Hum (%),Avg Rel Hum (%),Dew Point (F),Avg Wind Speed (mph),Avg Soil Temp (F),Year,County
0,2,FivePoints,1/1/2018,0.06,0.00,219.0,7.3,63.4,35.3,47.8,82.0,46.0,65.0,36.6,3.3,51.1,2018,Fresno
1,2,FivePoints,1/2/2018,0.04,0.00,127.0,7.4,59.8,37.7,47.2,80.0,52.0,67.0,36.7,3.1,51.3,2018,Fresno
2,2,FivePoints,1/3/2018,0.04,0.00,125.0,8.4,61.1,37.3,49.9,79.0,49.0,68.0,39.9,4.5,51.3,2018,Fresno
3,2,FivePoints,1/4/2018,0.07,0.01,219.0,11.6,69.2,48.7,56.8,94.0,52.0,74.0,48.5,5.8,53.0,2018,Fresno
4,2,FivePoints,1/5/2018,0.07,0.00,239.0,12.7,73.8,47.5,59.8,94.0,49.0,72.0,50.8,4.2,54.4,2018,Fresno


In [23]:
env_df = env_df.groupby([ "County","Year"]).agg({
    "ETo (in)":"sum",
    "Precip (in)":"sum",
    "Sol Rad (Ly/day)" : "mean",
    "Avg Vap Pres (mBars)": "mean",
    "Max Air Temp (F)":"max",
    "Min Air Temp (F)":"min",
    "Avg Air Temp (F)":"mean",
    "Max Rel Hum (%)":"max",
    "Min Rel Hum (%)":"min",
    "Avg Rel Hum (%)":"mean",
    "Dew Point (F)":"mean",
    "Avg Wind Speed (mph)":"mean",
    "Avg Soil Temp (F)":"mean"
    
}).reset_index()
env_df.head()

,County,Year,ETo (in),Precip (in),Sol Rad (Ly/day),Avg Vap Pres (mBars),Max Air Temp (F),Min Air Temp (F),Avg Air Temp (F),Max Rel Hum (%),Min Rel Hum (%),Avg Rel Hum (%),Dew Point (F),Avg Wind Speed (mph),Avg Soil Temp (F)
0,Alameda,2018,132.60,40.31,437.525880,11.453520,99.3,19.9,57.542340,100.0,9.0,69.966874,47.291408,3.950828,61.806315
1,Alameda,2019,141.87,57.87,430.446849,11.761336,101.6,26.8,57.896237,100.0,9.0,70.883349,47.977046,3.985042,60.736218
2,Alameda,2020,116.04,24.83,494.154263,11.959811,108.8,28.3,59.092828,100.0,8.0,69.369418,48.479296,4.134100,62.526928
3,Alameda,2021,492.58,165.70,404.350427,11.454701,92.1,29.5,58.760684,99.0,20.0,67.102564,47.108547,3.329060,62.196581
4,Alpine,2019,33.83,5.56,460.730570,5.764767,92.0,4.6,50.388601,99.0,9.0,46.461140,28.563731,7.091192,51.319171


In [24]:
# FILL IN MISSING COUNTIES WITH COMPARABLE COUNTIES

# [name for name in env_df.values() if name in ["Placer", "Shasta", "Butte", "Marin", "Humboldt", "Merced", "Stanislaus", 
#                             "San Joaquin", "Alpine","Colusa","Fresno", "Colusa"] ]
change_list = ["Placer", "Shasta", "Butte", "Marin", "Humboldt", "Merced", "Stanislaus", 
                            "San Joaquin", "Alpine","Colusa","Fresno"]

new_name_list = [
    "Sierra", "Lassen", "Plumas", "San Francisco", "Del Norte",  "Mariposa", "Tuolumne", "Calaveras",
    "Mono", "Glenn", "Madera"
]

temp_env_df = env_df[env_df['County'].isin(change_list)]


for (name, new_name) in zip(change_list, new_name_list):
   
    temp_env_df["County"] = temp_env_df["County"].replace([name], new_name)
    
    
    
temp_env_df = temp_env_df.append(env_df[env_df['County'] == "Shasta"]).append(env_df[env_df['County'] == "Colusa"]).append(env_df[env_df['County'] == "Placer"])
temp_env_df["County"] = temp_env_df["County"].replace(["Shasta"], "Trinity")
temp_env_df["County"] = temp_env_df["County"].replace(["Colusa"], "Lake")
temp_env_df["County"] = temp_env_df["County"].replace(["Placer"], "Nevada")

# temp_env_df
env_df = env_df.append(temp_env_df)


C:\Users\appii\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [26]:
# EXPORT THE ENVIRONMENTAL DATASET
env_df = env_df.dropna()
env_df.to_csv("Clean_Data/environment.csv")

In [23]:
# ****** DATASET 3 : THE ELECTRICITY USAGE DATA *******

electric_df = pd.read_csv("Data/ElectricityByCounty CALIFORNIA Usage in Millions of kWh.csv")

In [24]:
# Find the totals for each sector, and then drop that column.
electric_df = electric_df[electric_df["Sector"] == "Total"].drop(["Sector"], axis=1)

# Drop the years before 2013
electric_df = electric_df.iloc[:, :8]

In [25]:
electric_df.head()

,County,2019,2018,2017,2016,2015,2014,2013
2,ALAMEDA,10684.085867,10391.361826,11079.450563,10791.224841,10235.384987,10299.877787,10618.661255
5,ALPINE,18.906214,18.704208,18.976912,17.415654,16.209900,15.983364,18.176468
8,AMADOR,317.885054,304.092677,313.312866,309.094100,285.308438,289.501492,310.344276
11,BUTTE,1396.246344,1475.788821,1529.818607,1482.073760,1492.098630,1489.484147,1502.980505
14,CALAVERAS,330.559070,332.353511,347.931437,316.143601,311.620104,319.671411,326.656878


In [26]:
# EXPORT THE ELECTRIC DATA

electric_df = electric_df.dropna()
electric_df.to_csv("Clean_Data/electricity.csv")

In [42]:
# ****** DATASET 4 : THE HOUSING DATA *******

housing_df = pd.read_csv("Data/housing.csv")

In [40]:
housing_df = housing_df.drop(["housing_median_age", "total_rooms","total_bedrooms"], axis=1)
housing_df.head()

,Unnamed: 0,longitude,latitude,population,households,median_income,median_house_value,ocean_proximity,County
0,0,-124.35,40.54,806.0,270.0,3.0147,94600.0,NEAR OCEAN,NaN
1,1,-124.30,41.80,1298.0,478.0,1.9797,85800.0,NEAR OCEAN,NaN
2,2,-124.30,41.84,1244.0,456.0,3.0313,103600.0,NEAR OCEAN,NaN
3,3,-124.27,40.69,1194.0,465.0,2.5179,79000.0,NEAR OCEAN,NaN
4,4,-124.26,40.58,907.0,369.0,2.3571,111400.0,NEAR OCEAN,NaN


In [31]:
# GET THE COUNTY FROM THE LONG/LAT PAIR

In [32]:
housing_df = housing_df.groupby([ "longitude","latitude"]).agg({
    "population":"sum",
    "households":"sum",
    "median_income" : "mean",
    "median_house_value": "mean",
    "ocean_proximity":"min"
    
}).reset_index()


In [33]:
housing_df

,longitude,latitude,population,households,median_income,median_house_value,ocean_proximity
0,-124.35,40.54,806.0,270.0,3.0147,94600.0,NEAR OCEAN
1,-124.30,41.80,1298.0,478.0,1.9797,85800.0,NEAR OCEAN
2,-124.30,41.84,1244.0,456.0,3.0313,103600.0,NEAR OCEAN
3,-124.27,40.69,1194.0,465.0,2.5179,79000.0,NEAR OCEAN
4,-124.26,40.58,907.0,369.0,2.3571,111400.0,NEAR OCEAN
...,...,...,...,...,...,...,...
12585,-114.56,33.69,333.0,117.0,1.6509,85700.0,INLAND
12586,-114.55,32.80,1431.0,608.0,1.2750,56100.0,INLAND
12587,-114.49,33.97,83.0,45.0,1.6154,87500.0,INLAND
12588,-114.47,34.40,1129.0,463.0,1.8200,80100.0,INLAND


In [34]:
housing_df.iloc[[2]]

,longitude,latitude,population,households,median_income,median_house_value,ocean_proximity
2,-124.3,41.84,1244.0,456.0,3.0313,103600.0,NEAR OCEAN


In [ ]:
import googlemaps
def add_counties(housing_df):
    counties = []
    #     locator = Nominatim(user_agent="Google")
    gmaps = googlemaps.Client(API_KEY)
    
    i = 0
    for (long, lat) in housing_df[["longitude", "latitude"]].values: 
        try:
            county_name = [result["short_name"] for result in gmaps.reverse_geocode((lat, long))[0]["address_components"] if result["types"][0] == "administrative_area_level_2"][0]
            counties.append(county_name)
        except:
            counties.append("NULL")
        if i % 1000 == 0 and i > 1:
            print(i)
        #             pd.DataFrame(counties).to_csv("counties"+str(i)+".csv")
        #             counties = []
        i+=1
        
    housing_df["County"] = counties
    
add_counties(housing_df)


# coordinates = 37.88, -122.23

# remove the print statements, make the writes 

# location = locator.reverse(coordinates)
# location.raw

In [43]:
housing_df = housing_df.dropna()
housing_df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [44]:
# EXPORT THE HOUSING DATA

housing_df.to_csv("Clean_Data/housing.csv")

In [ ]:
# ANALYSIS TOOL TO LOOK AT OUTLIERS

def boxplot_it(df, column_number):
    print("THERE ARE {} NUMERICAL COLUMNS IN THIS DF AVAILABLE".format( len(df.select_dtypes(include='number').columns)) )
    number_df = pd.DataFrame(df.select_dtypes(include='number').iloc[:,column_number])
    [plt.boxplot(x=number_df[column].dropna(), labels=[column]) for column in number_df]

In [ ]:
boxplot_it(fire_df,4 )
